In [1]:
import torch
import torch.nn as nn
import numpy as np
import datasets.datasets as dtset
import utils.losses as losses

from utils.weakener import Weakener

In [2]:
Data = dtset.OpenML_Dataset('iris', batch_size = 16)
Weak = Weakener(Data.num_classes)
Weak.generate_M(model_class='pll',pll_p = 0.2)
train_X,train_y,test_X,test_y =  Data.get_data()
Weak.generate_weak(train_y) #z and w

c:\Users\danibacaicoa\vscode_projects\Learning_from_weak_labels\datasets\datasets.py:89: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  data = openml.datasets.get_dataset(openml_ids[self.dataset])


(tensor([2, 1, 4, 5, 3, 4, 3, 1, 2, 3, 3, 3, 0, 3, 0, 0, 0, 2, 3, 3, 0, 1, 5, 3,
         2, 1, 0, 1, 3, 2, 3, 1, 1, 5, 0, 2, 3, 3, 1, 3, 3, 5, 0, 1, 5, 3, 1, 1,
         4, 6, 0, 5, 4, 1, 0, 5, 0, 1, 1, 1, 2, 3, 3, 3, 2, 0, 5, 1, 0, 0, 2, 5,
         5, 0, 3, 3, 0, 1, 3, 0, 3, 0, 1, 6, 2, 1, 0, 5, 0, 0, 6, 1, 6, 4, 1, 1,
         2, 4, 0, 0, 0, 1, 4, 5, 1], dtype=torch.int32),
 tensor([[0., 1., 1.],
         [0., 1., 0.],
         [1., 0., 1.],
         [1., 1., 0.],
         [1., 0., 0.],
         [1., 0., 1.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 1., 1.],
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 0., 1.],
         [0., 1., 1.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [1., 1., 0.],
         [1., 0., 0.],
         [0., 1., 1.],
         [0., 1., 0.],
         [0., 0., 1.]

In [4]:
Weak.M

array([[0.  , 0.  , 0.64],
       [0.  , 0.64, 0.  ],
       [0.  , 0.16, 0.16],
       [0.64, 0.  , 0.  ],
       [0.16, 0.  , 0.16],
       [0.16, 0.16, 0.  ],
       [0.04, 0.04, 0.04]])

In [21]:
Weak.M.T.shape
Weak.virtual_matrix()

In [23]:
Weak.Y.shape

(3, 7)

In [24]:
Weak.M.shape

(7, 3)

In [9]:
print(Weak.M[1,:])
print(list(rng.dirichlet(Weak.M[1,:], 1).squeeze()))

[0.   0.64 0.  ]
[0.0, 0.9999999999999999, 0.0]


In [31]:

print(Weak.Y[1,:])
print(Weak.Y[:,1])
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) 
print(softmax(Weak.Y[:,1]))
print(list(rng.dirichlet(Weak.Y[:,1].T, 1).squeeze()))

[-0.13638621  1.25331976  0.59196748 -0.1281657  -0.13209072  0.55908541
  0.34267235]
[-0.12947809  1.25331976 -0.12384167]
[0.1668978  0.66526104 0.16784116]


ValueError: alpha < 0

In [42]:
# Tis is a solution with M
rng = np.random.default_rng()
V = np.array([list(rng.dirichlet(Weak.M[i,:], 1).squeeze()) for i in range(Weak.M.shape[0])]).T
print(V.shape)
print(V)
print(V.sum(0))


(3, 7)
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  9.85621186e-01 1.29687812e-01 9.99947485e-01]
 [0.00000000e+00 1.00000000e+00 7.17144314e-01 0.00000000e+00
  0.00000000e+00 8.70312188e-01 8.04887806e-16]
 [1.00000000e+00 0.00000000e+00 2.82855686e-01 0.00000000e+00
  1.43788140e-02 0.00000000e+00 5.25151649e-05]]
[1. 1. 1. 1. 1. 1. 1.]


In [43]:
#Solution with ones d dim
rng = np.random.default_rng()
V = np.array([list(rng.dirichlet(Weak.M[i,:]*0+1, 1).squeeze()) for i in range(Weak.M.shape[0])]).T
print(V.shape)
print(V)
print(V.sum(0))

(3, 7)
[[0.33496888 0.79582505 0.31097093 0.10189125 0.14122115 0.25008379
  0.02287904]
 [0.13260223 0.0216276  0.10675193 0.25249424 0.41000445 0.3598183
  0.12606039]
 [0.5324289  0.18254735 0.58227714 0.64561451 0.4487744  0.3900979
  0.85106057]]
[1. 1. 1. 1. 1. 1. 1.]


In [65]:
#Solution with ones d dim
rng = np.random.default_rng()
V = np.array([list(rng.dirichlet(np.ones(Weak.c), 1).squeeze()) for i in range(Weak.d)]).T
print(V.shape)
print(V)
print(V.sum(0))

(3, 7)
[[0.06360351 0.41266791 0.058639   0.49727095 0.18405588 0.70895758
  0.27247874]
 [0.75639284 0.47683664 0.40264348 0.42761466 0.27562489 0.22385876
  0.29590054]
 [0.18000366 0.11049545 0.53871751 0.0751144  0.54031923 0.06718366
  0.43162072]]
[1. 1. 1. 1. 1. 1. 1.]


In [68]:
rng = np.random.default_rng()
V = np.array([list(rng.dirichlet(np.ones(Weak.d), 1).squeeze()) for i in range(Weak.d)]).T
print(V.shape)
print(V)
print(V.sum(0))

(7, 7)
[[0.02040501 0.21735273 0.12118465 0.24438943 0.03733655 0.09539013
  0.16250649]
 [0.16730682 0.09432863 0.10856293 0.06795872 0.37264874 0.14790287
  0.13718985]
 [0.18635436 0.04494969 0.22379999 0.19126862 0.21178017 0.06171812
  0.03847162]
 [0.26364783 0.2375035  0.02408394 0.1300971  0.13015182 0.049353
  0.28661968]
 [0.03890103 0.05228233 0.16625365 0.0514707  0.13755637 0.11987205
  0.07452812]
 [0.07824343 0.13676282 0.2915184  0.22639147 0.01273587 0.07969569
  0.24698433]
 [0.24514152 0.2168203  0.06459645 0.08842395 0.09779048 0.44606814
  0.05369991]]
[1. 1. 1. 1. 1. 1. 1.]


In [59]:
np.ones(h)

array([1., 1., 1., 1., 1.])

In [60]:
h = int((Weak.d+Weak.c)/2)
V = np.array([list(rng.dirichlet(np.ones(h), 1).squeeze()) for i in range(Weak.d)]).T
print(V.shape)
print(V)
print(V.sum(0))

(5, 7)
[[0.12760681 0.17115709 0.35028456 0.27296201 0.13750216 0.05098009
  0.10927753]
 [0.14287498 0.04452647 0.13298154 0.24711947 0.48189361 0.17063529
  0.0299103 ]
 [0.04166078 0.33742248 0.20583285 0.05008354 0.06812096 0.11793808
  0.63806911]
 [0.40952847 0.01910471 0.13115539 0.11649068 0.18953081 0.40513969
  0.0463956 ]
 [0.27832896 0.42778925 0.17974566 0.31334429 0.12295247 0.25530685
  0.17634747]]
[1. 1. 1. 1. 1. 1. 1.]


In [49]:
print(V.shape)
print(np.linalg.matrix_rank(V))
print(np.sum(V,0))

(10, 1023)
10
[3.53974430e-09 2.94175909e-03 1.49892451e-03 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [ ]:
A = V_ini@Weak.M
A_new = V_ini/(A@one)

: 

In [ ]:
A_new@Weak.M@one

: 

In [ ]:
A_new

: 

In [ ]:
np.min(A_new)

: 

: 